# Capstone Project - Find a Optimal location for a Indian Restarunet in Colombo, Sri Lanka

## Table of Contents
* [Introduction: Business Problem](#intro)
* [Data](#data)
* [Methodology](#method)
* [Analysis](#analysis)
* [Results and Discussion](#result)
* [Conclusion](#conclusion)


## 1.Introduction: Business Problem <a class="anchor" id="intro"></a>

### 1.1 Backgound
The success of establishing a new restaurant depends on several factors: demand, brand loyalty, quality of food, competition, ambiance,etc... In most cases, a restaurant’s location plays an essential determinant for its success. Hence, it is advantageous and of utmost importance to determine the most strategic location for establishment in order to maximize business profits.

### 1.2 Business Problem
A client seeks to establish a franchised Indian restaurant,in a Colombo District,Sri Lanka. Which neighbourhood would appear to be the optimal and most strategic location for the business operations?
The objective of this capstone project is to locate the optimal neighborhood for operation.since there are lots of resttarunts in Colombo , we will try to detect location sthat are not already crowded wtih resturants.we are also particulary intersted ares with no Indian restaruants in vicinity, but this could be a place very near to other venues such as parks, Movie Theators etc..

### 1.3 Intersets
Fellow entrepreneurs seeking to either establish a new restaurant of a certain niche or have plans to expand their franchised restaurants would be very interested in the competitive advantages and business values this finding can potentially reap.

## 2.Data <a class="anchor" id="data"></a>

### 2.1 Creating list of Neighbourhoods in Colombo
Colombo is cpatial of sri lanka with 676km^2 land area with population over 2.5 million.Colombo districted divided in to 557 Grama Niladhari (**GN**) Divisions , which would be the smallest division for the neigbourhood). [The survey department of srilanka ](https://it.survey.gov.lk/gn_updating/) contains map of GN divisions of sri lanka and from this site we can download KMZ file of Colombo district. whith a help of onine conervertor ,data can be converted to [geojson file](https://github.com/dilancw/Coursera_Capstone/blob/main/COLOMBO.geojson).

lets import required libries for data processing and cleansing

In [9]:
#!pip install folium==0.5.0 

     |████████████████████████████████| 79 kB 7.4 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=9aacd53499a6ded3b52029569de1426c64dcaafb3b051a93ee91fdc141d4b299
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium


In [10]:
import requests
import json
import pandas as pd
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium

download colombo.geojson file

In [3]:
#!wget https://raw.githubusercontent.com/dilancw/Coursera_Capstone/main/COLOMBO.geojson

--2021-03-29 00:51:05--  https://raw.githubusercontent.com/dilancw/Coursera_Capstone/main/COLOMBO.geojson
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10228977 (9.8M) [text/plain]
Saving to: ‘COLOMBO.geojson’

COLOMBO.geojson     100%[===================>]   9.75M  --.-KB/s    in 0.09s   

2021-03-29 00:51:06 (108 MB/s) - ‘COLOMBO.geojson’ saved [10228977/10228977]



In [11]:
json_file_path = "COLOMBO.geojson"

with open(json_file_path, 'r') as j:
     contents = json.loads(j.read())


In [12]:
colombo_gn_list=[]
for item in  contents["features"]:
    gn_data={"Neighbourhood":None, "latitude":None, "longitude":None}
    #print(item["properties"]["Name"])
    coordinate = (item["geometry"]["coordinates"][0][0])
    lat = []
    long = []
    for pos in coordinate:
        lat.append(pos[1])
        long.append(pos[0])
    
    gn_data["Neighbourhood"] = item["properties"]["Name"]
    gn_data["latitude"] = sum(lat)/len(lat)
    gn_data["longitude"] =  sum(long)/len(long)
                                           
    colombo_gn_list.append(gn_data)

#print(colombo_gn_list)
colombo_df =pd.DataFrame(colombo_gn_list,columns=['Neighbourhood','latitude','longitude']) 


Lets explore the pandas data frame

In [13]:
colombo_df.head()

,Neighbourhood,latitude,longitude
0,Nawagampura,6.945493,79.877171
1,Soysapura South,6.803476,79.882984
2,Slave Island,6.925610,79.850186
3,Salawa,6.934586,80.120860
4,Madulawa North,6.823405,80.062936


let see number of divisions

In [14]:
colombo_df.describe()

,latitude,longitude
count,557.000000,557.000000
mean,6.865926,79.971055
std,0.056088,0.094684
min,6.721204,79.843199
25%,6.823197,79.894628
50%,6.863584,79.940052
75%,6.916087,80.029040
max,6.977882,80.219566


lets mark these neihbourhoods in folium map

In [15]:
# create map of New York using latitude and longitude values
colombo_latitude = 6.86
colombo_longitude = 79.97
map_colombo = folium.Map(location=[colombo_latitude, colombo_longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(colombo_df['latitude'], colombo_df['longitude'],colombo_df['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_colombo)  
    
map_colombo

Saved image of map to view  on github
![Colmobo Neighbourhood Map]("colombo.PNG")


### 2.2 Pulling Restaurant / Food venues in Colombo respect to each neighborhood

In [34]:
CLIENT_ID = 'X4EAV0OCCZJQLHJVDQZ0EQLNNGVPUAEXPG02KJCY4UMWKWU1' # your Foursquare ID
CLIENT_SECRET = 'WYUELIK2YY4QWOPBKKD4KUEKAF1Z0FNEQSBBQW20ELG1VT2W' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [35]:
def getNearbyVenues(names, latitudes, longitudes, cat_id, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            cat_id)
            
        # make the GET request
       # print(url)
        response= requests.get(url)
        #print(response)
        results =response.json()["response"]['groups'][0]['items']
        count_venues = response.json()["response"]['totalResults']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            count_venues,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        print(name,"-->",count_venues)
        #break
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'NeighborhoodLatitude', 
                  'NeighborhoodLongitude',
                  'Venue_Count',           
                  'Venue', 
                  'VenueLatitude', 
                  'VenueLongitude', 
                  'VenueCategory']
    
    return(nearby_venues)

In [38]:
FOOD = "4d4b7105d754a06374d81259"
#colombo_venues = getNearbyVenues(names=colombo_df['Neighbourhood'],
#                                   latitudes=colombo_df['latitude'],
#                                   longitudes=colombo_df['longitude'], cat_id=FOOD)
#colombo_venues.to_csv("cmb_venues.csv")

Since there is a limit per hour for api calls to foursqueare the comobo_venues were saves to a .csv file and use for later processings.

In [44]:
colombo_venues = pd.read_csv("https://raw.githubusercontent.com/dilancw/Coursera_Capstone/main/cmb_venues.csv",index_col= 0)
colombo_venues.head()

,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,Venue_Count,Venue,VenueLatitude,VenueLongitude,VenueCategory
0,Nawagampura,6.945493,79.877171,1,Anton Ground Floor Cafeteria,6.941537,79.879230,Cafeteria
1,Soysapura South,6.803476,79.882984,4,Pawana Restaurant,6.805076,79.884362,Restaurant
2,Soysapura South,6.803476,79.882984,4,perera & sons,6.805549,79.884559,Bakery
3,Soysapura South,6.803476,79.882984,4,Friends Bakers,6.805638,79.885281,Snack Place
4,Soysapura South,6.803476,79.882984,4,Shanghai Hotel,6.800471,79.881562,Chinese Restaurant


Lets map the Neighborhoods where it has at least 8 food venues. As part of data cleaning.

In [60]:
min_food_venues = 8
colombo_venues_grouped = colombo_venues.groupby(["Neighborhood"]).mean()
colombo_high_rank= colombo_venues_grouped[colombo_venues_grouped["Venue_Count"] > 8]
colombo_high_rank.reset_index(inplace = True)
print("There are",colombo_high_rank.shape[0]," Neighbourhoods which have more than",min_food_venues," food places")

There are 35  Neighbourhoods which have more than 8  food places


Lets map this famous neighborhood for foods venues using folium. The marker size dipits the number of food places in that neighbourhood

In [63]:
# create map of New York using latitude and longitude values
colombo_latitude = 6.86
colombo_longitude = 79.97
map_colombo = folium.Map(location=[colombo_latitude, colombo_longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood,count in zip(colombo_high_rank['NeighborhoodLatitude'], colombo_high_rank['NeighborhoodLongitude'],colombo_high_rank['Neighborhood'],colombo_high_rank['Venue_Count']):
    label = '{},{}'.format(neighborhood,count)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=count/2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_colombo)  
    
map_colombo

Saved image of map to view  on github
![Colmobo Food Venues Map]("rank_food.PNG")

we will use colombo_high_rank pandas dataframe as potential foodie places around colombo

In [64]:
colombo_high_rank.head()

,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,Venue_Count,VenueLatitude,VenueLongitude
0,Asiri Uyana,6.893244,79.930635,12,6.891192,79.928667
1,Bambalapitiya,6.900776,79.853752,32,6.900378,79.853908
2,Borella South,6.914662,79.880874,9,6.913773,79.877716
3,Dehiwala East,6.856360,79.869340,9,6.855583,79.867404
4,Dehiwala West,6.857379,79.861998,14,6.857489,79.862681


Lets check the indian restaranut venues

In [68]:
colombo_indian_restaruent_venues = colombo_venues[colombo_venues["VenueCategory"].str.contains("Indian")]
colombo_indian_restaruent_venues

,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,Venue_Count,Venue,VenueLatitude,VenueLongitude,VenueCategory
13,Maligakanda,6.928678,79.868292,9,Al-Maas Family Restaurant,6.927558,79.867738,Indian Restaurant
23,Kotahena West,6.949052,79.860551,10,Sri Vaishnavi Vihar,6.949131,79.862597,Indian Restaurant
32,Kotahena West,6.949052,79.860551,10,Hotel Sooriya,6.945195,79.859758,Indian Restaurant
74,Galle Face,6.923542,79.845544,15,Alhambra,6.920585,79.847758,Indian Restaurant
186,Karagampitiya,6.848020,79.871462,14,Kawdana Kada,6.847766,79.868602,Indian Restaurant
...,...,...,...,...,...,...,...,...
1252,Wickramasinghapura,6.882442,79.939683,2,Kottu Spot - Hokandara,6.878876,79.941447,Indian Restaurant
1316,Thalangama North B,6.905897,79.950737,8,Lemongrass Malabe,6.905470,79.949800,Indian Restaurant
1317,Thalangama North B,6.905897,79.950737,8,Spice Rack,6.906302,79.946821,Indian Restaurant
1352,Boralesgamuwa West C,6.843216,79.896206,5,Thiru Murugan's Food Waves,6.844801,79.898184,South Indian Restaurant


lets use colombo_high_rank and colombo_indian_restaruent_venues to find out the best place to setup next indian resturant.

## Methodology <a class="anchor" id="method"></a>

## Analysis <a class="anchor" id="analysis"></a>

## Results and Discussion <a class="anchor" id="result"></a>

## Conclusion <a class="anchor" id="conclusion"></a>